https://surprise.readhedocs.io/en/stable/

In [34]:
import surprise 

print(surprise.__version__)

1.1.1


### Surprise 를 이용한 추천 시스템 구축

In [35]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [36]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

In [37]:
algo = SVD()
algo.fit(trainset) 

In [38]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.7744264573448683, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.661007607465865, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.0711589875193175, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.5817588673629253, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.2117149837909036, details={'was_impossible': False})]

In [39]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.7744264573448683),
 ('882', '291', 3.661007607465865),
 ('535', '507', 4.0711589875193175)]

In [40]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.29   {'was_impossible': False}


In [41]:
accuracy.rmse(predictions)

RMSE: 0.9476


0.9476379142469752

### Surprise 주요 모듈 소개

In [42]:
import pandas as pd

ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('./dataset/ml-latest-small/ratings_noh.csv', index=False, header=False)


In [59]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [43]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./dataset/ml-latest-small/ratings_noh.csv',reader=reader)

In [44]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

In [50]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
print(accuracy.rmse(predictions))
[(pred.uid,pred.iid,pred.est) for opresd in predictions[:3]]


RMSE: 0.8682
0.8681952927143516


[('196', '302', 4.292710058654728),
 ('196', '302', 4.292710058654728),
 ('196', '302', 4.292710058654728)]

[과제] ratings.csv 데이터로 SVD 클래스를 이용하여 아래와 같이 추천알고리즘을 빌딩하고 평가하세요.

- 교차 검증 및 하이퍼파라미터 튜닝 : measures(RMSE, MAE), cv = 5, param = n_epoch(20,40,60), n_factors(50,100,200)

In [46]:
# 20초 소요
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8741  0.8797  0.8709  0.8777  0.8754  0.8756  0.0030  
MAE (testset)     0.6736  0.6768  0.6689  0.6751  0.6721  0.6733  0.0027  
Fit time          3.46    3.58    3.60    3.55    3.56    3.55    0.05    
Test time         0.12    0.12    0.24    0.12    0.23    0.17    0.06    


{'test_rmse': array([0.87411354, 0.87973863, 0.8709279 , 0.87765106, 0.87542215]),
 'test_mae': array([0.67361616, 0.67678518, 0.66888215, 0.67507926, 0.67213371]),
 'fit_time': (3.4572789669036865,
  3.577802896499634,
  3.59500789642334,
  3.5487961769104004,
  3.5605311393737793),
 'test_time': (0.11702704429626465,
  0.11602592468261719,
  0.2380530834197998,
  0.12302780151367188,
  0.23305225372314453)}

In [47]:
# 3분 소요 ,n_jobs=-1 하면 30초 걸렸음
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3,n_jobs=-1)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8766481573967582
{'n_epochs': 20, 'n_factors': 50}


[과제] 개인화 영화 추천 시스템을 작성하세요.(잠재요인 협업필터링 방식)

- 데이터셋 : ratings_noh.csv, movies.csv

In [52]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./dataset/ml-latest-small/ratings_noh.csv', reader=reader)

# train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
# 전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()


In [63]:
data_folds.__dict__

{'reader': <surprise.reader.Reader at 0x1417f007100>,
 'has_been_split': False,
 'ratings_file': './dataset/ml-latest-small/ratings_noh.csv',
 'raw_ratings': [('1', '1', 4.0, '964982703'),
  ('1', '3', 4.0, '964981247'),
  ('1', '6', 4.0, '964982224'),
  ('1', '47', 5.0, '964983815'),
  ('1', '50', 5.0, '964982931'),
  ('1', '70', 3.0, '964982400'),
  ('1', '101', 5.0, '964980868'),
  ('1', '110', 4.0, '964982176'),
  ('1', '151', 5.0, '964984041'),
  ('1', '157', 5.0, '964984100'),
  ('1', '163', 5.0, '964983650'),
  ('1', '216', 5.0, '964981208'),
  ('1', '223', 3.0, '964980985'),
  ('1', '231', 5.0, '964981179'),
  ('1', '235', 4.0, '964980908'),
  ('1', '260', 5.0, '964981680'),
  ('1', '296', 3.0, '964982967'),
  ('1', '316', 3.0, '964982310'),
  ('1', '333', 5.0, '964981179'),
  ('1', '349', 4.0, '964982563'),
  ('1', '356', 4.0, '964980962'),
  ('1', '362', 5.0, '964982588'),
  ('1', '367', 4.0, '964981710'),
  ('1', '423', 3.0, '964982363'),
  ('1', '441', 4.0, '964980868'),
  

In [53]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)


In [54]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./dataset/ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])


사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [55]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)


user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [56]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [58]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ round(pred.est,1) for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.3
Star Wars: Episode IV - A New Hope (1977) : 4.3
Pulp Fiction (1994) : 4.3
Silence of the Lambs, The (1991) : 4.2
Godfather, The (1972) : 4.2
Streetcar Named Desire, A (1951) : 4.2
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.1
Star Wars: Episode VI - Return of the Jedi (1983) : 4.1
Goodfellas (1990) : 4.1
Glory (1989) : 4.1
